In [969]:
import numpy as np
from tqdm import tqdm
from time import sleep

In [970]:
class NN():

    def __init__(self, hiddenLayer_neuron = 2,learning_rate = 0.1):

        self.data = np.array([[0,0],
                              [0,1],
                              [1,0],
                              [1,1]])
        self.target = np.array([[0],[1],[1],[0]])

        self.hl_neuron = hiddenLayer_neuron
        
        self.w_hidden = np.random.uniform(size=(2,self.hl_neuron))
        self.w_out = np.random.uniform(size=(self.hl_neuron,1))

        self.b_hidden = np.random.uniform(size=(1,self.hl_neuron))
        self.b_out = np.random.uniform(size=(1,1))

        self.z0s = []
        self.z1s = []

        self.lr = learning_rate

        print("Init: ")
        self.print_weights()

    def ReLU(self,x):
        return x * (x > 0) 
    
    def bcel(self, y, y_hat):
        return -(y*np.log(y_hat) + (1-y) * np.log(1-y_hat))
    
    def drv_loss_start(self, y, y_hat):
        return 1 / (1-y_hat) if y == 0 else -1 / y_hat

    def drv_loss(self, y, y_hat):
        return y_hat-y
    
    def drv_sigmoid(self, z):
        return self.sigmoid(z) * (1-self.sigmoid(z))
    
    '''def drv_sigmoid(self, sigmoid_value):
        return sigmoid_value * (1-sigmoid_value)'''
    
    def drv_ReLU(self, z):
        return 1 * (z > 0)

    def sigmoid(self,x):
        return 1/(1 + np.exp(-x))
    
    def execute(self,x):
        result = self.sigmoid(self.forward_pass(x)[0])
        return np.round(result)
    
    def forward_pass(self,x):
        x = np.array(x)
        z0 = np.dot(x.T,self.w_hidden) + self.b_hidden
        hidden = self.ReLU(z0)
        z1 = np.dot(hidden,self.w_out) + self.b_out
        return z1, z0
    

    def backward_pass(self,x,y):

        alpha = 0.1
        z1, z0 = self.z1s.pop(), self.z0s.pop()

        y_hat = self.sigmoid(z1)
        d1 = self.drv_loss_start(y,y_hat)*self.drv_sigmoid(z1)
        d0 = self.drv_loss(y,y_hat) * np.dot(self.drv_ReLU(z0),self.w_out)

        self.w_out -= alpha* np.dot(self.ReLU(z0).T,d1) 
        self.w_hidden -= alpha* np.dot(x.reshape(2,1),d0)

        self.b_out -= alpha*np.sum(d1, axis=0, keepdims=True)
        self.b_hidden -= alpha*np.sum(d0, axis=0, keepdims=True)
        
    def print_weights(self):
        print("W_hidden: " + str(self.w_hidden)) 
        print("B_hidden: " + str(self.b_hidden)) 
        print("W_out: " + str(self.w_out)) 
        print("B_out: " + str(self.b_out), end="\n\n")


    def accuracy(self):
        counter = 0
    
        for i in range(len(self.data)):
            if(self.execute(self.data[i]) == self.target[i]):
                counter += 1
        loss = 0
        for i in range(len(self.data)):
            loss += self.bcel(self.target[i],self.sigmoid(self.forward_pass(self.data[i])[0]))

        return counter / len(self.data), loss / len(self.data)
     
    def train(self,epochs, batch_size, fast_train = False):

        if epochs > 0:
            for epoch in range(epochs):
                index = np.random.randint(4,size=batch_size)
                for i in index:
                    z1, z0 = self.forward_pass(self.data[i])
                    self.z0s.append(z0)
                    self.z1s.append(z1)

                self.z1s.reverse()
                self.z0s.reverse()

                if not fast_train:

                    for i in tqdm(index,desc="Epoch: " + str(epoch)): 
                        self.backward_pass(self.data[i],self.target[i])
                        sleep(.1)
                    acc, loss =  self.accuracy()
                    print("Current accuracy: " +f"{acc*100}%")
                    print("Current loss: " +f"{loss}")
                
                else:
                    for i in index:
                        self.backward_pass(self.data[i],self.target[i])
                    acc, loss =  self.accuracy()
            
            print("Current accuracy: " +f"{acc*100}%")
            print("Current loss: " +f"{loss}")

        
        else:
            epoch = 0
            acc = 0
            while acc  != 1:
                index = np.random.randint(4,size=batch_size)
                for i in index:
                    z1, z0 = self.forward_pass(self.data[i])
                    self.z0s.append(z0)
                    self.z1s.append(z1)

                self.z1s.reverse()
                self.z0s.reverse()

                if not fast_train:

                    for i in tqdm(index,desc="Epoch: " + str(epoch)): 
                        self.backward_pass(self.data[i],self.target[i])
                        sleep(.1)
                    acc, loss =  self.accuracy()
                    print("Current accuracy: " +f"{acc*100}%")
                    print("Current loss: " +f"{loss}")
                
                else:
                    for i in index:
                        self.backward_pass(self.data[i],self.target[i])
                    acc, loss =  self.accuracy()
                    print("Current accuracy: " +f"{acc*100}%")
                    print("Current loss: " +f"{loss}")
                    
                epoch += 1
        print("Epochs trained: " + str(epoch+1))
        self.print_weights()
        print(acc)
        return acc       

In [971]:
nn = NN(4,learning_rate=0.000001)

Init: 
W_hidden: [[0.49826721 0.14350359 0.74227182 0.7431287 ]
 [0.55730448 0.87642334 0.10319849 0.93610544]]
B_hidden: [[0.4139112  0.97901778 0.77606971 0.45474128]]
W_out: [[0.61796742]
 [0.69178443]
 [0.06718173]
 [0.01635982]]
B_out: [[0.42972515]]



In [972]:
nn.train(epochs = -1,batch_size= 2,fast_train=True)

Current accuracy: 50.0%
Current loss: [[0.75290828]]
Current accuracy: 50.0%
Current loss: [[0.74890648]]
Current accuracy: 50.0%
Current loss: [[0.7434422]]
Current accuracy: 50.0%
Current loss: [[0.81687169]]
Current accuracy: 50.0%
Current loss: [[0.7341685]]
Current accuracy: 50.0%
Current loss: [[0.70770345]]
Current accuracy: 50.0%
Current loss: [[0.69489813]]
Current accuracy: 50.0%
Current loss: [[0.69527212]]
Current accuracy: 50.0%
Current loss: [[0.71406011]]
Current accuracy: 50.0%
Current loss: [[0.69572809]]
Current accuracy: 50.0%
Current loss: [[0.70026392]]
Current accuracy: 50.0%
Current loss: [[0.70141019]]
Current accuracy: 75.0%
Current loss: [[0.69721004]]
Current accuracy: 75.0%
Current loss: [[0.69861948]]
Current accuracy: 50.0%
Current loss: [[0.69599021]]
Current accuracy: 50.0%
Current loss: [[0.70247681]]
Current accuracy: 50.0%
Current loss: [[0.72796479]]
Current accuracy: 50.0%
Current loss: [[0.70898934]]
Current accuracy: 50.0%
Current loss: [[0.742791

1.0

In [973]:
print(nn.execute([0,0]))
print(nn.execute([0,1]))
print(nn.execute([1,0]))
print(nn.execute([1,1]))

[[0.]]
[[1.]]
[[1.]]
[[0.]]
